In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU, Conv2D
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import SpatialDropout1D, Bidirectional, Flatten, BatchNormalization, LSTMCell, RNN, GRU
import tensorflow as tf
%matplotlib inline 

In [ ]:
# Максимальное количество слов 
num_words = 10000
# Максимальная длина новости
max_news_len = 30
# Количество классов новостей
nb_classes = 4

## Загрузка набора данных

Загружаем данные для обучения

In [ ]:
from sklearn.utils import shuffle

num_words = 40000
max_news_len = 10


text_train = pd.read_csv("train_new.csv").dropna().text
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

## Сверточная нейронная сеть

In [ ]:
model_cnn = Sequential()
model_cnn.add(Embedding(num_words, 124, input_length=max_news_len))
model_cnn.add(LSTM(128, activation='relu'))
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))


In [ ]:
from tensorflow.keras.metrics import Precision, Recall
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model_cnn.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', Recall(name='Recall'), Precision(name='Precision'), f1_metric])

In [ ]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 124)           4960000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               129536    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 5,106,177
Trainable params: 5,106,177
Non-trainable params: 0
_________________________________________________________________


### Загрузим модель

In [ ]:
# model_cnn.save_weights('SASHA_my_model_weights.h5')
model_cnn.load_weights('SASHA_my_model_weights.h5')

### Метрика

In [ ]:
from sklearn.metrics import f1_score

test_sequences = tokenizer.texts_to_sequences(text_test)	
x_test = pad_sequences(test_sequences, maxlen=max_news_len, value=0, padding='post')

label_predict = model_cnn.predict_classes(x_test)

print('Оценка F1', f1_score(label_predict, label_test.to_list()))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Оценка F1 0.36750483558994196


### Для предсказания 

In [ ]:
test_sequences = tokenizer.texts_to_sequences(text_test)	
x_test = pad_sequences(test_sequences, maxlen=max_news_len, value=0, padding='post')
label_predict = model_cnn.predict_classes(x_test)